# Diffusion Tensor Imaging

### Import Library

In [2]:
import pydicom
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from skimage.io import imread
import seaborn as sns
from glob import glob
import ipywidgets as widgets
import scipy.ndimage as ndimage
import imageio
from skimage.segmentation import clear_border
from skimage import measure
from skimage.measure import label,regionprops
from scipy import ndimage as ndi
from scipy.ndimage import measurements, center_of_mass, binary_dilation, zoom
import plotly.graph_objects as go
import math

## Define the Mosaic dicom picture fiel and path in .ima format 

In [3]:
##  Mosaic Picturess  ####

mosaic_pictures_files = "/Applications/tez/tez_data/TOMRUK_ZAHIDE_DTI/DTI/DIFFUSION_HIGHRES_0009/" # Our input 
mosaic_paths = os.listdir(mosaic_pictures_files)
mosaic_paths.sort() # Bu sort işlemi aslında çok öneli çünkü biz bu kodu her çalıştırdığımızda aynı sıralamayı veriyor ve bize bir nevi referans oluyor.
nSeries = len(mosaic_paths)
nSeries

71

## Slice Information

In [4]:
mosaic_picture = pydicom.read_file(mosaic_pictures_files + mosaic_paths[10])
a = mosaic_picture.fix_meta_info


In [5]:

def shape_calculation_slice():
    
    mHeight = mosaic_picture.Rows
    mHeight = int(mHeight)
    mWeight = mosaic_picture.Columns
    mWeight = int(mWeight)


    slice_number = mosaic_picture[0x019, 0x100a]
    slice_number = str(slice_number)
    index = slice_number.find(":")
    slice_number = slice_number[index+2:]
    slice_number = int(slice_number)

    height = int(mHeight / math.sqrt(slice_number))
    weight =int(mWeight / math.sqrt(slice_number))

    return slice_number, height, weight



In [6]:
slice_number, height, weight = shape_calculation_slice()


slice_number, height, weight

(64, 128, 128)

### Spliting Mosaic pictures

In [7]:
def split_slice(path):
    """
    Split the only one mosaic pictures
    """
    liste = []
    b_values = []
    mosaic_picture = pydicom.read_file(path)
    b_val = mosaic_picture[0x019, 0x100c] # B değerini veriyor ama sayısal veri vermiyor 
    b_values.append(b_val)
    mosaic_ndarray = mosaic_picture.pixel_array
    
    for i in range(8):
        for j in range(8):
            slice = mosaic_ndarray[i*128:(i*128)+128,j*128:(j*128)+128]
            liste.append(slice)
            
    return  liste, b_values




In [8]:

def all_mosaic_split(mosaic_pictures_files):
    """
    Split the all mosaic slices in given files
    """ 

    mosaic_paths = os.listdir(mosaic_pictures_files)
    mosaic_paths.sort()

    all_mosaic_slices = []
    b_values = []

    for k in mosaic_paths:
        mosaic_slices, b_val = split_slice(mosaic_pictures_files + k)
        str_b_value = str(b_val)
        index = str_b_value.find("'")
        values_int = int(str_b_value[index+1:len(str_b_value)-2])
        mosaic_slices = np.array(mosaic_slices)
        all_mosaic_slices.append(mosaic_slices)
        b_values.append(values_int)
    
    b_values = np.array(b_values)
    all_mosaic_slices = np.array(all_mosaic_slices)

    return all_mosaic_slices, b_values


In [9]:
all_mosaic_slices, b_values = all_mosaic_split(mosaic_pictures_files)

### Burda Zahide Tomruk için kendimiz elle b0 değerlerini ayırdık çünkü elde ettiğimiz b0 verileri liste içinde elde edebildik bilgisiyarda bunu ayıramadık

### B0 ve DWI ların  ayrılması


In [10]:
def b_values_divider(all_mosaic_slices, b_values):

    b0_index = []
    b0_mosaic_slicess = []
    b0_mosaic_slicess = np.array(b0_mosaic_slicess)

    dwi_index = []
    dwi_mosaic_slicess = []
    dwi_mosaic_slicess = np.array(dwi_mosaic_slicess)

    num = 0

    for i in b_values:

        if i == 0:
            b0_index.append(num)
        elif i > 0:
            dwi_index.append(num)
        else:
            print("Hatalı Değer B_values negatif bir değer alamaz")
            
        
        num = num + 1
    
    b0_index = np.array(b0_index)
    dwi_index = np.array(dwi_index)

    b0_mosaic_slicess = all_mosaic_slices[b0_index]
    dwi_mosaic_slicess = all_mosaic_slices[dwi_index]

    return b0_mosaic_slicess, dwi_mosaic_slicess, (b0_index,dwi_index)

In [11]:
b0_mosaic_slicess, dwi_mosaic_slicess, (b0_index,dwi_index) = b_values_divider(all_mosaic_slices, b_values)

In [12]:
b0_mosaic_slicess, dwi_mosaic_slicess, index = b_values_divider(all_mosaic_slices, b_values)

#### Index
Burdaki index verilerimiz bizim sırasıyla aldığımız path den dicom infolarına göre b değerlerinin hangi indexlerde 0 hangilerinde difüzyon olduğunu beliritiyor.

In [13]:
index

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 70]),
 array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
        27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
        44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
        61, 62, 63, 64, 65, 66, 67, 68, 69]))

### B0 değerlerinin ortalamasını alma

In [14]:
def b0_avarager(b0_slicess):
    # all b=0 values mosaic picturees average 
    # 4 dimentional data take average from [11,64,128,128] to [1,64,128,128]
    
    average_slices = [] 

    for i in range(len(b0_slicess[1])):
        img = np.zeros((128,128))
        
        for k in range(len(b0_slicess)):
            b0_img = b0_slicess[k,i,:,:]
            img = b0_img + img

        img = img / len(b0_slicess) 
        average_slices.append(img)


    average_slices = np.array(average_slices)

    return average_slices

In [15]:
b0_average_mosaic_slice = b0_avarager(b0_mosaic_slicess)

In [16]:
m0,m1,m2 = b0_average_mosaic_slice.shape
# Add a slider that starts with 0 and ends at the number of
# slices along the axial plane, n0=99.
@widgets.interact(axial_slice=(0,m0-1))
# Define the function that shows the images of the specified slice number.
# It starts with the 10th slice. And you can scroll over any slice
# using the slider.

def axial_slicer(axial_slice=50):
  
  # Show the image of the specified slice number in 'gray' color-map
  # and axial aspect ratio
  
  plt.imshow(b0_average_mosaic_slice[axial_slice,:,:])
  #plt.imshow(mosaic_pictures_vol[axial_slice,:,:], cmap='hot')
  #plt.imshow(mosaic_pictures_vol[axial_slice,:,:], cmap='RdBu')


  # Don't show the axis
  plt.axis('off')
  plt.title("Average b=0 Brain Slices")

interactive(children=(IntSlider(value=50, description='axial_slice', max=63), Output()), _dom_classes=('widget…

In [ ]:
data = {"Mosaic_Paths":mosaic_paths, "Mosaic_slices": list(all_mosaic_slices), "B_values":b_values, "Gradient_Infromation":range(len(mosaic_paths))}
information_dataFrame = pd.DataFrame(data)
information_dataFrame